# 安装transformers, datsets

In [1]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 7.9 MB/s 
     |████████████████████████████████| 3.3 MB 51.2 MB/s 
     |████████████████████████████████| 895 kB 50.2 MB/s 
     |████████████████████████████████| 636 kB 63.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install datasets

     |████████████████████████████████| 264 kB 9.1 MB/s 
     |████████████████████████████████| 76 kB 4.8 MB/s 
     |████████████████████████████████| 243 kB 52.9 MB/s 
     |████████████████████████████████| 118 kB 52.6 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


# 3.3.1 Overview

视频学习

In [ ]:
# 前期准备+复习之前的代码
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer)

# 新的内容
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2) # 这里多加了一个num_labels参数

from transformers import TrainingArguments
# training_args = TrainingArguments("test-trainer")
training_args = TrainingArguments(
    "test-trainer",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    learning_rate=2e-5,
    weight_decay=0.01,
)

# Trainer API，训练过程
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer
)
trainer.train()

# evaluate过程（在validation集合上）—— 方式1，在训练完成后进行一次evaluation
import numpy as np
from datasets import load_metric

predictions = trainer.predict(tokenized_datasets["train"])
metric = load_metric("glue", "mrpc")
preds = np.argmax(predictions.predictions, axis=-1)
print("training set上的准确率和F1值是：", metric.compute(predictions=preds, references=predictions.label_ids))

predictions = trainer.predict(tokenized_datasets["validation"])
metric = load_metric("glue", "mrpc")
preds = np.argmax(predictions.predictions, axis=-1)
print("validation set上的准确率和F1值是：", metric.compute(predictions=preds, references=predictions.label_ids))

predictions = trainer.predict(tokenized_datasets["test"])
metric = load_metric("glue", "mrpc")
preds = np.argmax(predictions.predictions, axis=-1)
print("test set上的准确率和F1值是：", metric.compute(predictions=preds, references=predictions.label_ids))

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}


Step,Training Loss
500,0.401000
1000,0.086200


Saving model checkpoint to test-trainer/checkpoint-500
Configuration saved in test-trainer/checkpoint-500/config.json
Model weights saved in test-trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-500/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-1000
Configuration saved in test-trainer/checkpoint-1000/config.json
Model weights saved in test-trainer/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-1000/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, idx, sentence1.
***** Running Prediction *****
  Num exampl

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, idx, sentence1.
***** Running Prediction *****
  Num examples = 408
  Batch size = 16


training set上的准确率和F1值是： {'accuracy': 0.9972737186477645, 'f1': 0.9979806138933763}


The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, idx, sentence1.
***** Running Prediction *****
  Num examples = 1725
  Batch size = 16


validation set上的准确率和F1值是： {'accuracy': 0.8529411764705882, 'f1': 0.8958333333333334}
test set上的准确率和F1值是： {'accuracy': 0.8510144927536232, 'f1': 0.8909630886720407}


In [ ]:
# 前期准备+复习之前的代码
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer)

# 新的内容，model
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2) # 这里多加了一个num_labels参数

# 新的内容，指定训练的超参数TrainingArguments类
from transformers import TrainingArguments
# training_args = TrainingArguments("test-trainer") # 这个是基础版，可以在这个上边加参数的
training_args = TrainingArguments(
    "test-trainer",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    learning_rate=2e-5,
    weight_decay=0.01,
    evaluation_strategy="epoch" # 注意这里做了一些修改

)

# Trainer API，训练过程，evaluate过程，在训练过程中监控 （epoch）
from transformers import Trainer
from datasets import load_metric
import numpy as np

metric = load_metric("glue", "mrpc")
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()


predictions = trainer.predict(tokenized_datasets["train"])
metric = load_metric("glue", "mrpc")
preds = np.argmax(predictions.predictions, axis=-1)
print("training set上的准确率和F1值是：", metric.compute(predictions=preds, references=predictions.label_ids))

predictions = trainer.predict(tokenized_datasets["validation"])
metric = load_metric("glue", "mrpc")
preds = np.argmax(predictions.predictions, axis=-1)
print("validation set上的准确率和F1值是：", metric.compute(predictions=preds, references=predictions.label_ids))

predictions = trainer.predict(tokenized_datasets["test"])
metric = load_metric("glue", "mrpc")
preds = np.argmax(predictions.predictions, axis=-1)
print("test set上的准确率和F1值是：", metric.compute(predictions=preds, references=predictions.label_ids))

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.361646,0.857843,0.902357
2,No log,0.347513,0.857843,0.900000
3,0.394500,0.513989,0.865196,0.906303
4,0.394500,0.647855,0.860294,0.901893
5,0.083800,0.619927,0.862745,0.902439


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, idx, sentence1.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, idx, sentence1.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 16
Saving model checkpoint to test-trainer/checkpoint-500
Configuration saved in test-trainer/checkpoint-500/config.json
Model weights saved in test-trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, idx, 

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, idx, sentence1.
***** Running Prediction *****
  Num examples = 408
  Batch size = 16


training set上的准确率和F1值是： {'accuracy': 0.9948200654307524, 'f1': 0.996170126990526}


The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, idx, sentence1.
***** Running Prediction *****
  Num examples = 1725
  Batch size = 16


validation set上的准确率和F1值是： {'accuracy': 0.8627450980392157, 'f1': 0.902439024390244}
test set上的准确率和F1值是： {'accuracy': 0.8382608695652174, 'f1': 0.8811248402215593}


自己的再次尝试+代码整理

In [8]:
from datasets import load_dataset, load_metric # 加载数据集和度量方法
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification # 从ckpt加载tokenizer和模型
from transformers import TrainingArguments, Trainer # 训练
import numpy as np
import torch

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# 准备传入trainer的各项参数
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2) # 这里之前还多了一个num_labels参数
training_args = TrainingArguments(
    "test-trainer",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    learning_rate=2e-5,
    weight_decay=0.01,
    evaluation_strategy="epoch" # 每个epoch都会测试一下
)

# 准备evaluate的一些过程
metric = load_metric("glue", "mrpc")
def compute_metrics(eval_preds):
    ''' 应该是对一个batch进行evaluate '''
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# 构建一个trainer对象
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train() # 开始进行训练

# 训练后的预测(训练集，其它同理)
predictions = trainer.predict(tokenized_datasets["train"])
metric = load_metric("glue", "mrpc")
preds = np.argmax(predictions.predictions, axis=-1)
print("train set上的准确率和F1值是：", metric.compute(predictions=preds, references=predictions.label_ids))




Reusing dataset glue (/root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}


  0%|          | 0/1 [00:00<?, ?ba/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-f5ffdad84b81b183.arrow
loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 2,
  "

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.370534,0.835784,0.888147
2,No log,0.360796,0.845588,0.893039
3,0.395500,0.555082,0.843137,0.888112
4,0.395500,0.702418,0.848039,0.893836
5,0.090100,0.726040,0.848039,0.891608


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, idx, sentence1.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, idx, sentence1.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 16
Saving model checkpoint to test-trainer/checkpoint-500
Configuration saved in test-trainer/checkpoint-500/config.json
Model weights saved in test-trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, idx, 

train set上的准确率和F1值是： {'accuracy': 0.9970010905125409, 'f1': 0.9977782266208847}
